<a href="https://colab.research.google.com/github/timothyalcaide/darknet/blob/master/yolo_darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Link Codlab, Drive and Github

[source tuto](https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228)

In [29]:
# Mount Google Drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')        

/content/gdrive/My Drive/IA
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [30]:
# Clone github repository setup
from os.path import join  

ROOT = "/content/gdrive" 
MY_GOOGLE_DRIVE_PATH = 'My Drive/IA/' 
GIT_USERNAME = "timothyalcaide" 
GIT_TOKEN = "7802dd553c96b9dfa6becd8141de1a14340e6629"  
GIT_REPOSITORY = "darknet" 

PROJECTS_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
PROJECT_PATH = join(PROJECTS_PATH, GIT_REPOSITORY)

# It's good to print out the value if you are not sure 
print("PROJECTS_PATH: ", PROJECTS_PATH)   
print("PROJECT_PATH: ", PROJECT_PATH)   

GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECTS_PATH:  /content/gdrive/My Drive/IA/
PROJECT_PATH:  /content/gdrive/My Drive/IA/darknet
GIT_PATH:  https://7802dd553c96b9dfa6becd8141de1a14340e6629@github.com/timothyalcaide/darknet.git


In [33]:
%cd "{PROJECTS_PATH}" 
%rm -rf darknet
!git clone "{GIT_PATH}"

/content/gdrive/My Drive/IA
Cloning into 'darknet'...
remote: Enumerating objects: 14263, done.
remote: Total 14263 (delta 0), reused 0 (delta 0), pack-reused 14263
Receiving objects: 100% (14263/14263), 12.62 MiB | 5.32 MiB/s, done.
Resolving deltas: 100% (9771/9771), done.
Checking out files: 100% (2014/2014), done.


In [ ]:
!git config --global user.email "timothy.alcaide@gmail.com"
!git config --global user.name "Timothy Alcaide"

In [ ]:
%cd "{PROJECTS_PATH}" 
!git add .
!git commit -m "add codelab, drive, github config"
!git push 

In [ ]:
# Mettre à jour le repo drive par rapport au *distant*
!git pull --rebase 